In [ ]:
import codecs
import os
import sys
import threading
import serial
from serial.tools.list_ports import comports

1. Gimbal control

In [1]:
from a2gmeasurements import GimbalRS2
import time, timeit
import numpy as np

myGimbal = GimbalRS2()
myGimbal.start_thread_gimbal()


[DEBUG]: Gimbal thread started


Finding required time for gimbal to answer a position request

In [ ]:
from a2gmeasurements import GimbalRS2
import time, timeit
import numpy as np

myGimbal = GimbalRS2()
myGimbal.start_thread_gimbal()

assumed_time_for_execute_time = 0.0015 # s
gimbal_speed = (1800/3.47) # DEG/s
condition_continue = True
tol = gimbal_speed * assumed_time_for_execute_time
cnt = 0
myGimbal.request_current_position()
time.sleep(0.0015)
last_yaw = myGimbal.yaw 
last_roll = myGimbal.roll
print('Outside: ',last_yaw, last_roll)
myGimbal.setPosControl(yaw=1250, roll=0, pitch=0)
time.sleep(1)
while(condition_continue):
    print('Execution time: ', timeit.default_timer() - start)
    myGimbal.request_current_position()
    time.sleep(0.0015)
    print('Inside: ', myGimbal.yaw, myGimbal.roll)
    
    if np.abs(last_yaw - myGimbal.yaw) < tol and np.abs(last_roll - myGimbal.roll) < tol:
        condition_continue = False
    else:
        last_yaw = myGimbal.yaw
        last_roll = myGimbal.roll
        start = timeit.default_timer()
    cnt = cnt+1
    print(cnt)
    


In [3]:
#myGimbal.request_current_position()
myGimbal.setPosControl(yaw=0, roll=0, pitch=0)

True

Exception in thread Thread-4 (receive):
Traceback (most recent call last):
  File "/home/jvvtt64/mambaforge/envs/groundnode/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/home/jvvtt64/mambaforge/envs/groundnode/lib/python3.10/threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jvvtt64/Documents/a2gMeasurements/a2gmeasurements.py", line 463, in receive
    print("Start receiving messages")
  File "/home/jvvtt64/mambaforge/envs/groundnode/lib/python3.10/site-packages/can/bus.py", line 98, in recv
    msg, already_filtered = self._recv_internal(timeout=time_left)
  File "/home/jvvtt64/mambaforge/envs/groundnode/lib/python3.10/site-packages/can/interfaces/pcan/pcan.py", line 536, in _recv_internal
    raise PcanCanOperationError(self._get_formatted_error(result))
can.interfaces.pcan.pcan.PcanCanOperationError: The value of a PCAN-Hardware handle is invalid


In [ ]:
myGimbal.stop_thread_gimbal()
time.sleep(0.05)
myGimbal.actual_bus.shutdown()

2. Anritsu Control

In [ ]:
from a2gmeasurements import myAnritsuSpectrumAnalyzer

mySA = myAnritsuSpectrumAnalyzer()
mySA.spectrum_analyzer_connect()
rx_power =  mySA.retrieve_max_pow()
print(rx_power)
mySA.spectrum_analyzer_close()

3. Septentrio Control

In [ ]:
from a2gmeasurements import GpsSignaling
mySeptentrioGPS = GpsSignaling(DBG_LVL_2=True)

In [ ]:
mySeptentrioGPS.serial_connect()
mySeptentrioGPS.serial_instance.reset_input_buffer()


In [ ]:
#mySeptentrioGPS.sendCommandGps(cmd='sga, MultiAntenna') # by default this is the command
mySeptentrioGPS.sendCommandGps(cmd='sgd, ETRS89')
mySeptentrioGPS.sendCommandGps(cmd='spm, Rover, RTK')

In [ ]:
mySeptentrioGPS.sendCommandGps(cmd='setNTRIPSettings, NTR1, Client, opencaster.nls.fi, 2101, fuave, v5E3oKu853A, VRS-FKP')

In [ ]:

#mySeptentrioGPS.start_gps_data_retrieval(msg_type='NMEA', nmea_type='GGA', interval='sec1') # HDF instead of GGA for heading info
#mySeptentrioGPS.start_gps_data_retrieval(msg_type='NMEA', nmea_type='HDT', interval='sec1') # HDF instead of GGA for heading info

mySeptentrioGPS.start_gps_data_retrieval(msg_type='SBF', stream_number=1, interface='USB', interval='sec2')
mySeptentrioGPS.start_thread_gps()


In [ ]:
import struct
struct.calcsize('<1c3H1I1H2B3d5f1d1f4B2H1I2B4H1B')

In [ ]:
mySeptentrioGPS.stop_gps_data_retrieval(stream_number=1, msg_type='SBF')
#mySeptentrioGPS.stop_gps_data_retrieval(msg_type='SBF')
mySeptentrioGPS.stop_thread_gps()

In [ ]:
print(len(mySeptentrioGPS.SBF_frame_buffer), len(mySeptentrioGPS.NMEA_buffer))
if len(mySeptentrioGPS.SBF_frame_buffer) > 0:
    print(mySeptentrioGPS.SBF_frame_buffer[-1])
if len(mySeptentrioGPS.NMEA_buffer) > 0:
    print(mySeptentrioGPS.NMEA_buffer[-1])

In [ ]:
mySeptentrioGPS.sendCommandGps('sdio, USB1,, -SBF-NMEA')

In [ ]:
mySeptentrioGPS.serial_instance.close()

In [ ]:
mySeptentrioGPS.SBF_frame_buffer

In [ ]:
xsbf = mySeptentrioGPS.SBF_frame_buffer[-1]['X']
ybf = mySeptentrioGPS.SBF_frame_buffer[-1]['Y']
zbf = mySeptentrioGPS.SBF_frame_buffer[-1]['Z']

In [ ]:
lat=mySeptentrioGPS.NMEA_buffer[-1]['Latitude']
lon=mySeptentrioGPS.NMEA_buffer[-1]['Longitude']

In [ ]:
print(lat, lon)

Precision of distance when using planar coordinates ('precision' is given by google estimate of distance)

In [ ]:
import pyproj as proj
from a2gmeasurements import HelperA2GMeasurements
import numpy as np

pt_hammentie_50 = [60.186977663598164, 24.961182204780805]
pt_hakaniemi_mkt = [60.17907971164074, 24.951228689442726]

h = HelperA2GMeasurements()

lat_pln_hamm, lon_pln_hamm = h.convert_DDMMS_to_planar(24.961182204780805, 60.186977663598164)
lat_pln_hak, lon_pln_hak = h.convert_DDMMS_to_planar(24.951228689442726, 60.17907971164074)

# Compare this distance with Google MAPS: CHECKED ---> within 2m error
d_hamm_hak = np.linalg.norm(np.array([lon_pln_hamm, lat_pln_hamm]) - np.array([lon_pln_hak, lat_pln_hak]))

Example of a real GPS NMEA output

In [ ]:
real_gps_data = {'Timestamp': '110815.00', 
                 'Latitude': '6011.1011606', 
                 'Latitude Direction': 'N', 
                 'Longitude': '02448.9144475', 
                 'Longitude Direction': 'E', 
                 'GPS Quality Indicator': '2', 
                 'Number of Satellites in use': '28', 
                 'Horizontal Dilution of Precision': '0.6', 
                 'Antenna Alt above sea level (mean)': '0.9230', 
                 'Units of altitude (meters)': 'M', 
                 'Geoidal Separation': '19.5883', 
                 'Units of Geoidal Separation (meters)': 'M', 
                 'Age of Differential GPS Data (secs)': '1.8', 
                 'Differential Reference Station ID': '0123'}

real_gps_sbf_data = {'TOW': 476864000, 'WNc': 2237, 'MODE': 1, 'ERR': 0, 
                     'X': 2885666.3154522795, 'Y': 1334484.5807205061, 'Z': 5510775.657593954, 
                     'Undulation': 19.586938858032227, 
                     'Vx': -0.0005603009485639632, 'Vy': 0.013242623768746853, 'Vz': 0.0013509213458746672, 
                     'COG': -20000000000.0, 'RxClkBias': 0.1324483297937943, 'RxClkDrift': 0.019449610263109207, 
                     'TimeSystem': 0, 'Datum': 0, 'NrSV': 12, 'WACorrInfo': 0, 'ReferenceID': 65535, 
                     'MeanCorrAge': 65535, 'SignalInfo': 807534857, 'AlertFlag': 1, 'NrBases': 0, 
                     'PPPInfo': 0, 'Latency': 64, 'HAccuracy': 1500, 'VAccuracy': 1471}

# Estos datos fueron tomados del Septentrio colocado en la ventana de mi oficina en Micronova
real_gps_sbf_data_1 = {'ID': 'Coordinates',
  'TOW': 118960000,
  'WNc': 2251,
  'MODE': 1,
  'ERR': 0,
  'X': 2885668.616751026,
  'Y': 1334491.9205545222,
  'Z': 5510766.505916461,
  'Datum': 0}

real_gps_sbf_data_2 = {'ID': 'Coordinates',
  'TOW': 118962000,
  'WNc': 2251,
  'MODE': 1,
  'ERR': 0,
  'X': 2885668.675170647,
  'Y': 1334491.5537603002,
  'Z': 5510766.218123834,
  'Datum': 0}

real_gps_sbf_data_3 = {'ID': 'Heading',
  'TOW': 118960000,
  'WNc': 2251,
  'ERR': 0,
  'MODE': 2,
  'Heading': 169.2756805419922,
  'Pitch': -0.01572183519601822,
  'Roll': -20000000000.0}

In [ ]:
import a2gUtils as ag

lat, lon, h = ag.geocentric2geodetic(real_gps_sbf_data_2['X'], real_gps_sbf_data_2['Y'], real_gps_sbf_data_2['Z'])

print(lat, lon, h)

X, Y, Z = ag.geodetic2geocentric(lat, lon, h)

print(X, Y, Z)

Server connection

In [ ]:
from socket import socket
import time
from a2gmeasurements import HelperA2GMeasurements

host = 'localhost'
myHelper = HelperA2GMeasurements('GROUND', host)
myHelper.HelperStartA2GCom()


In [ ]:
myHelper.HelperA2GStopCom()
print(myHelper.SOCKET_BUFFER, len(myHelper.SOCKET_BUFFER))

5. Signal generator

In [ ]:
import pyvisa

rm = pyvisa.ResourceManager()
print(rm.list_resources())

inst = rm.open_resource('GPIB0::19::INSTR')
print(inst.query("*IDN?"))

inst.write('F0 28.95 GH\n')
inst.write('L0 20 DM\n')

# Turn On RF output
inst.write('RF1\n')
# Turn Off RF output
inst.write('RF0\n')

In [ ]:
from a2gmeasurements import myAnritsuSpectrumAnalyzer

dir = 'C://Users//jvjulian//OneDrive - Teknologian Tutkimuskeskus VTT//Documents//Measurement 1//20221012133438//'
file = 'CONT20221012133504968.rsm'

anritsu = myAnritsuSpectrumAnalyzer()
anritsu.parse_xml_file(dir + file)
anritsu.XML_file

In [ ]:
from PyQt6.QtWidgets import QApplication, QLabel, QWidget
import sys

app = QApplication([])

window = QWidget()
window.setWindowTitle("PyQt App")
window.setGeometry(100, 600, 280, 280) # 1st: x coordinate, 2nd: y coordinate, 3rd: width, 4th: height
helloMsg = QLabel("<h1>Hello World</h1>", parent=window)
helloMsg.move(60, 15)
window.show()

sys.exit(app.exec())


In [ ]:
import datetime
import time
import sys

today_date = datetime.date.today()
today_date = [int(i) for i in today_date.strftime("%Y-%m-%d").split('-')]

ex = '110815.00'

print(today_date)

complete_date = datetime.datetime(year=today_date[0], 
                                  month=today_date[1], 
                                  day=today_date[2], 
                                  hour=int(ex[0:2]), 
                                  minute=int(ex[2:4]), 
                                  second=int(ex[4:6]))

a = time.mktime(complete_date.timetuple())
print(a)

print(time.time())

Animation to prove gimbal_follows_drone function does its job

In [ ]:
from pyproj import Transformer, Geod
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw
from a2gUtils import GPSVis
from a2gmeasurements import HelperA2GMeasurements

upper_left = {'LAT': 60.1871, 'LON': 24.8113}
lower_right = {'LAT': 60.1837, 'LON': 24.8245}

hi_q = {'LAT': 60.18592, 'LON': 24.81174 }
fut_hub = {'LAT': 60.18650, 'LON': 24.81350}                        
fat_liz = {'LAT': 60.18555, 'LON': 24.82041}
aalto_metro = {'LAT':60.18495, 'LON': 24.82302}

micronova_office = {'LAT': 60.18463111366192, 'LON': 24.818124782978057}

wgs84_geod = Geod(ellps='WGS84')

transproj_1 = Transformer.from_crs({"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'}, "EPSG:4326", always_xy=True,)
transproj_2 = Transformer.from_crs("EPSG:4326", {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'})

x1, y1 = transproj_2.transform(fat_liz['LAT'], fat_liz['LON'])
x2, y2 = transproj_2.transform(aalto_metro['LAT'], aalto_metro['LON'])

az_fat_aalto,_, dist_fat_aalto = wgs84_geod.inv(fat_liz['LON'], fat_liz['LAT'], aalto_metro['LON'], aalto_metro['LAT'])
az_fat_hiq,_, dist_fat_hiq = wgs84_geod.inv(fat_liz['LON'], fat_liz['LAT'], hi_q['LON'], hi_q['LAT'])
az_fat_futhub,_, dist_fat_futhub = wgs84_geod.inv(fat_liz['LON'], fat_liz['LAT'], fut_hub['LON'], fut_hub['LAT'])
az_aalto_hiq,_, dist_aalto_hiq = wgs84_geod.inv(aalto_metro['LON'], aalto_metro['LAT'], hi_q['LON'], hi_q['LAT'])
az_aalto_futhub,_, dist_aalto_futhub = wgs84_geod.inv(aalto_metro['LON'], aalto_metro['LAT'], fut_hub['LON'], fut_hub['LAT'])
az_hiq_futhub,_, dist_hiq_futhub = wgs84_geod.inv(hi_q['LON'], hi_q['LAT'], fut_hub['LON'], fut_hub['LAT'])

'''
print(az_fat_aalto, dist_fat_aalto)
print(az_fat_hiq, dist_fat_hiq)
print(az_fat_futhub, dist_fat_futhub)
print(az_aalto_hiq, dist_aalto_hiq)
print(az_aalto_futhub, dist_aalto_futhub)
print(az_hiq_futhub, dist_hiq_futhub)
'''

heading = -10
myHelper = HelperA2GMeasurements(ID='GROUND', SERVER_ADDRESS='0.0.0.0')

yaw_hiq, pitch_hiq = myHelper.gimbal_follows_drone(heading=heading, lat_ground=micronova_office['LAT'], lon_ground=micronova_office['LON'], height_ground=2, lat_drone=hi_q['LAT'], lon_drone=hi_q['LON'], height_drone=20)
yaw_futhub, pitch_futhub = myHelper.gimbal_follows_drone(heading=heading, lat_ground=micronova_office['LAT'], lon_ground=micronova_office['LON'], height_ground=2, lat_drone=fut_hub['LAT'], lon_drone=fut_hub['LON'], height_drone=20)
yaw_fatliz, pitch_fatliz = myHelper.gimbal_follows_drone(heading=heading, lat_ground=micronova_office['LAT'], lon_ground=micronova_office['LON'], height_ground=2, lat_drone=fat_liz['LAT'], lon_drone=fat_liz['LON'], height_drone=20)
yaw_aaltometro, pitch_aalto_metro = myHelper.gimbal_follows_drone(heading=heading, lat_ground=micronova_office['LAT'], lon_ground=micronova_office['LON'], height_ground=2, lat_drone=aalto_metro['LAT'], lon_drone=aalto_metro['LON'], height_drone=20)

print('HiQ: YAW: ' + str(yaw_hiq) + ', PITCH: ' + str(pitch_hiq))
print('FutHub: YAW: ' + str(yaw_futhub) + ', PITCH: ' + str(pitch_futhub))
print('FatLiz: YAW: ' + str(yaw_fatliz) + ', PITCH: ' + str(pitch_fatliz))
print('AaltoMetro: YAW: ' + str(yaw_aaltometro) + ', PITCH: ' + str(pitch_aalto_metro))

In [ ]:
vis = GPSVis(data_path='test_coordinates_map_micronova.csv',
             map_path='test_map_micronova.png',  # Path to map downloaded from the OSM.
             points=(upper_left['LAT'], upper_left['LON'], lower_right['LAT'], lower_right['LON'])) # Two coordinates of the map (upper left, lower right)

vis.create_image(color=(0, 0, 255), width=3, heading=heading)  # Set the color and the width of the GNSS tracks.
vis.plot_map(output='-')

In [ ]:
vis = GPSVis(points=(upper_left['LAT'], upper_left['LON'], lower_right['LAT'], lower_right['LON'])) # Two coordinates of the map (upper left, lower right)

vis.update_map_marker((hi_q['LAT'], hi_q['LON']), r=10)
vis.update_map_marker((fut_hub['LAT'], fut_hub['LON']), r=10)
vis.update_map_marker((fat_liz['LAT'], fat_liz['LON']), r=10)
vis.update_map_marker((aalto_metro['LAT'], aalto_metro['LON']), r=10)
vis.plot_map(output='-')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter
from a2gUtils import AngleAnnotation
from a2gmeasurements import HelperA2GMeasurements


myHelper = HelperA2GMeasurements(ID='GROUND', SERVER_ADDRESS='')

xlim = (0, 10)
ylim = (0, 10)

lon_ground_planar = (xlim[0] + xlim[1])/2
lat_ground_planar = 3
height_ground = 1

fig = plt.figure(figsize=((5, 5)))
ax = fig.add_subplot(111, aspect='equal', xlim=xlim, ylim=ylim)

ax.grid()


line1, = ax.plot([],[], lw=2, color='blue')
line2, = ax.plot([],[], lw=2, color='red')
line3, = ax.plot([],[], lw=1, color='gray', linestyle='--')
heading_angle_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, color='red')
yaw2set_text = ax.text(0.02, 0.65, '', transform=ax.transAxes, color='green')
angle_drone_wrt_horizontal_text = ax.text(0.02, 0.35, '', transform=ax.transAxes, color='blue')

def animate(i):
    lon_drone_planar = np.random.rand()*10
    lat_drone_planar = np.random.rand()*10
    
    heading = np.random.rand()*np.pi*2
      
    yaw, _, alpha = myHelper.ground_gimbal_follows_drone(heading=heading, 
                                                   lat_ground=lat_ground_planar, 
                                                   lon_ground=lon_ground_planar, 
                                                   height_ground=1, 
                                                   lat_drone=lat_drone_planar, 
                                                   lon_drone=lon_drone_planar, 
                                                   height_drone=10, 
                                                   coord_type='planar')
    
    line1.set_data([lon_drone_planar, lon_ground_planar], 
                  [lat_drone_planar, lat_ground_planar])
    
    if heading > np.pi:
        heading_wrt_long_axs = np.pi/2 - (heading - np.pi*2)
        if heading_wrt_long_axs > np.pi:
            heading_wrt_long_axs = heading_wrt_long_axs - np.pi*2
        elif heading_wrt_long_axs < -np.pi:
            heading_wrt_long_axs = heading_wrt_long_axs + np.pi*2
    else:
        heading_wrt_long_axs = np.pi/2 - heading
        if heading_wrt_long_axs > np.pi:
            heading_wrt_long_axs = heading_wrt_long_axs - np.pi*2
        elif heading_wrt_long_axs < -np.pi:
            heading_wrt_long_axs = heading_wrt_long_axs + np.pi*2
    
    x_heading = lon_ground_planar + 2*np.cos(heading_wrt_long_axs)
    y_heading = lat_ground_planar + 2*np.sin(heading_wrt_long_axs)
    
    line2.set_data([x_heading, lon_ground_planar], 
                   [y_heading, lat_ground_planar])
        
    heading_angle_text.set_text('Heading: %.1f' % np.rad2deg(heading))
    yaw2set_text.set_text('Yaw (' +  r"$\theta$"  + '): '  +  f"{float(yaw)/10}")
    angle_drone_wrt_horizontal_text.set_text('Drone angle (' + r"$\alpha$"  + '): ' + f"{int(np.rad2deg(alpha))}")
    
    if yaw > 0:
        am1 = AngleAnnotation((lon_ground_planar, lat_ground_planar), 
                          (lon_drone_planar, lat_drone_planar), (x_heading, y_heading),  
                          ax=ax, size=125, text=r"$\theta$", color='green', text_kw=dict(color="green"))
    else:
        am1 = AngleAnnotation((lon_ground_planar, lat_ground_planar), 
                          (x_heading, y_heading), (lon_drone_planar, lat_drone_planar), 
                          ax=ax, size=125, text=r"$\theta$", color='green', text_kw=dict(color="green"))
        
    if alpha > 0:        
        am2 = AngleAnnotation((lon_ground_planar, lat_ground_planar), 
                          (xlim[1], lat_ground_planar), (lon_drone_planar, lat_drone_planar),
                          ax=ax, size=65, text=r"$\alpha$", color='blue', text_kw=dict(color="blue"))
    else:
        am2 = AngleAnnotation((lon_ground_planar, lat_ground_planar), 
                          (lon_drone_planar, lat_drone_planar), (xlim[1], lat_ground_planar),
                          ax=ax, size=65, text=r"$\alpha$", color='blue', text_kw=dict(color="blue"))
    
    line3.set_data([xlim[1], lon_ground_planar], 
                   [lat_ground_planar, lat_ground_planar])
    
    return line1, line2, heading_angle_text, yaw2set_text

ani = animation.FuncAnimation(fig, animate, frames=10, interval=5, repeat=False)

ani.save('basic_animation_2.gif', dpi=300, writer=PillowWriter(fps=0.2))
#plt.show()

In [ ]:
from pyproj import CRS, Transformer

a = {'X': 2885666.3154522795, 'Y': 1334484.5807205061, 'Z': 5510775.657593954}

geodet_crs = CRS.from_epsg(4979) # Geodetic (lat,lon,h) system
geocent_crs = CRS.from_epsg(4978) # Geocentric (X,Y,Z) system

# pyproj transformer object from geocentric to geodetic
geocent_to_geodet = Transformer.from_crs(geocent_crs, geodet_crs)


lat_lon = geocent_to_geodet.transform(a['X'],a['Y'],a['Z'])

print(lat_lon)

In [ ]:
import datetime
gps_data = {}
gps_data['Timestamp'] = ''
for i in datetime.datetime.utcnow().timetuple()[3:6]:
    tmp = str(i)
    if len(tmp) == 1:
        tmp = '0' + tmp
    gps_data['Timestamp'] = gps_data['Timestamp'] + tmp
gps_data['Timestamp'] = float(int(gps_data['Timestamp']))
print(gps_data)

In [ ]:
import json
a = {'CMD_SOURCE': 'SETGIMBAL', 'TYPE': 'CMD', 'DATA': {'YAW': 1245.5, 'PITCH': -614}}
a = json.dumps(a)


b = json.loads(a)
b['DATA']['YAW'] 
     

In [ ]:
import time  
import threading
from a2gmeasurements import RepeatTimer

cnt = 0

def display(msg):  
    global cnt
    print(msg + ' ' + time.strftime('%H:%M:%S'))  
    cnt = cnt + 1
  
##We are now creating a thread timer and controling it  
timer = RepeatTimer(0.007,display,['Repeating'])  
timer.start() #recalling run  
print('Threading started')  
time.sleep(7)#It gets suspended for the given number of seconds  
print('Threading finishing')  
timer.cancel()
print(cnt)

In [ ]:
81*0.007

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

nfft = 1024
sc = 200

fd = np.zeros((nfft,), dtype='complex')
fd[(nfft >> 1) + sc] = 1
fd = np.fft.fftshift(fd, axes=0)

# Convert the waveform to time-domain
td = np.fft.ifft(fd, axis=0)

# Normalize the signal
td /= np.max([np.abs(td.real), np.abs(td.imag)])

print(td[0:10])

fig = plt.figure(figsize=((10, 5)))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.grid()
ax2.grid()

line1, = ax1.plot(fd, lw=2, color='blue')
line2, = ax2.plot(td, lw=2, color='red')

plt.show()


In [ ]:
import numpy as np

nfft = 1024
sc_min = -100
sc_max = 100
mod = 'qam'
seed = 100

np.random.seed(seed)
qam = (1 + 1j, 1 - 1j, -1 + 1j, -1 - 1j)  # QAM symbols

# Create the wideband sequence in frequency-domain
fd = np.zeros((nfft,), dtype='complex')
if mod == 'qam':
    fd[((nfft >> 1) + sc_min):((nfft >> 1) + sc_max)] = np.random.choice(qam, len(range(sc_min, sc_max)))
else:
    fd[((nfft >> 1) + sc_min):((nfft >> 1) + sc_max)] = 1

fd = np.fft.fftshift(fd, axes=0)

# Convert the waveform to time-domain
td = np.fft.ifft(fd, axis=0)

# Normalize the signal
td /= np.max([np.abs(td.real), np.abs(td.imag)])

print(type(td), td.shape)

fig = plt.figure(figsize=((10, 5)))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.grid()
ax2.grid()

line1, = ax1.plot(fd, lw=2, color='blue')
line2, = ax2.plot(td, lw=2, color='red')

plt.show()

In [ ]:
import plotly.express as px
import numpy as np 

theta = np.linspace(10, 30, 3)
dNLoS = np.linspace(20, 100, 9)

X, Y = np.meshgrid(theta, dNLoS)
Z = np.deg2rad(X)*Y # elementwise

fig = px.imshow(Z,
                labels=dict(x="", y="dNLoS [m]", color="hDrone"),
                x=[str(i) for i in theta],
                y=[str(i) for i in dNLoS]
               )
fig.update_xaxes(side="top")
#fig.show()

# Define dragmode, newshape parameters, amd add modebar buttons
fig.update_layout(
    dragmode='drawrect',
    newshape=dict(line_color='cyan'))
fig.show(config={'modeBarButtonsToAdd':['drawline',
                                        'drawopenpath',
                                        'drawclosedpath',
                                        'drawcircle',
                                        'drawrect',
                                        'eraseshape'
                                       ]})

In [ ]:
st = 0b1 
w1 = 0x0F
ep = 1
stop_bits = 0b00

st = 0b1 
w2 = 0xF8
ep = 0
stop_bits = 0b00

st = 0b1 
w3 = 0x3F
ep = 1
stop_bits = 0b00

st = 0b1 
w4 = 0x07
ep = 0
stop_bits = 0b00

st = 0b1 
w5 = 0x2B
ep = 1
stop_bits = 0b00

st = 0b1 
w6 = 0xFC
ep = 1
stop_bits = 0b00

st = 0b1 
w7 = 0x1B
ep = 1
stop_bits = 0b00

st = 0b1 
w8 = 0x71
ep = 1
stop_bits = 0b00

st = 0b1 
w9 = 0x95
ep = 1
stop_bits = 0b00

st = 0b1 
w10 = 0xED
ep = 1
stop_bits = 0b00

st = 0b1 
w11 = 0xC7
ep = 0
stop_bits = 0b00

st = 0b1 
w12 = 0xC1
ep = 0
stop_bits = 0b00

st = 0b1 
w13 = 0xF2
ep = 1
stop_bits = 0b00

st = 0b1 
w14 = 0xB2
ep = 0
stop_bits = 0b00

st = 0b1 
w15 = 0xDB
ep = 1
stop_bits = 0b00

st = 0b1 
w16 = 0xE0
ep = 0 
stop_bits = 0b00

st = 0b1 
w17 = 0xFC
ep = 1
stop_bits = 0b00

st = 0b1 
w18 = 0x1F
ep = 0
stop_bits = 0b00

st = 0b1 
w19 = 0x82
ep = 0
stop_bits = 0b00

st = 0b1 
w20 = 0x3E
ep = 1
stop_bits = 0b00

st = 0b1 
w21 = 0x0F
ep = 1
stop_bits = 0b00

st = 0b1 
w22 = 0xC1
ep = 0
stop_bits = 0b00

st = 0b1 
w23 = 0xFF
ep = 1
stop_bits = 0b00

st = 0b1 
w24 = 0xFF
ep = 1
stop_bits = 0b00

In [ ]:
# From FrSky and Oscilloscope
data1 = {'A': -100, 'E':3, 'T':-100, 'R': -2,
        '1': 0x0F, '2': 0x12, '3': 0xFF, '4': 0x07,
        '5': 0x2B, '6': 0xFC, '7': 0x1F, '8': 0x83,
        '9':0x95, '10': 0xF2, '11': 0xBE, '12': 0x57,
        '13': 0xA6, '14': 0xA2, '15': 0xDB, '16': 0xE0,
        '17': 0xFC, '18': 0x1F, '19': 0x83, '20': 0xF0,
        '21': 0x7E, '22': 0x0F, '23': 0xC1, '24': 0xFF, '25': 0xFF}

data2 = {'A': -50, 'E':3, 'T':-100, 'R': -2,
        '1': 0x0F, '2': 0xFA, '3': 0xBF, '4': 0x07,
        '5': 0x2B, '6': 0xFC, '7': 0x1F, '8': 0x83,
        '9':0x95, '10': 0xF2, '11': 0xBE, '12': 0x57,
        '13': 0xA6, '14': 0xA2, '15': 0xDB, '16': 0xE0,
        '17': 0xFC, '18': 0x1F, '19': 0x83, '20': 0xF0,
        '21': 0x7E, '22': 0x0F, '23': 0xC1, '24': 0xFF, '25': 0xFF}

data3 = {'A': 50, 'E':3, 'T':-100, 'R': -2,
        '1': 0x0F, '2': 0x5F, '3': 0x5F, '4': 0x07,
        '5': 0x2B, '6': 0xFC, '7': 0x1F, '8': 0x83,
        '9':0x95, '10': 0xF2, '11': 0xBE, '12': 0x57,
        '13': 0xA6, '14': 0xA2, '15': 0xDB, '16': 0xE0,
        '17': 0xFC, '18': 0x1F, '19': 0x83, '20': 0xF0,
        '21': 0x7E, '22': 0x0F, '23': 0xC1, '24': 0xFF, '25': 0xFF}

data4 = {'A': 100, 'E':3, 'T':-100, 'R': -2,
        '1': 0x0F, '2': 0xB7, '3': 0x1F, '4': 0x07,
        '5': 0x2B, '6': 0xFC, '7': 0x1F, '8': 0x83,
        '9':0x95, '10': 0xF2, '11': 0xBE, '12': 0x57,
        '13': 0x66, '14': 0xA2, '15': 0xDB, '16': 0xE0,
        '17': 0xFC, '18': 0x1F, '19': 0x83, '20': 0xF0,
        '21': 0x7E, '22': 0x0F, '23': 0xC1, '24': 0xFF, '25': 0xFF}

Oscilloscope X8R , Start Logic 1, Stop Logic 0, Even Logic 1 (when even 1s in data), DATA as seen from left to right in oscilloscope

6 ms between packets, SA (channel 5) in middle position

In [ ]:
data1 = {'A': 0, 'E':0, 'T':-100, 'R': 0,
        '1': 0x0F, '2': 0b11000100, '3': 0b00111111, '4': 0b00000111,
        '5': 0b00101011, '6': 0b11111100, '7': 0b00011111, '8': 0b10000011,
        '9':0b10010101, '10': 0b11110010, '11': 0b10111110, '12': 0b01010111,
        '13': 0b11111000, '14': 0b00111000, '15': 0b00100111, '16': 0b11100000,
        '17': 0b11111100, '18': 0b00011111, '19': 0b10000011, '20': 0b11110000,
        '21': 0b01111110, '22': 0b00001111, '23': 0b11000001, '24': 0xFF, '25': 0xFF}

data1 = {'A': -99, 'E':0, 'T':-100, 'R': 0,
        '1': 0x0F, '2': 0b01110010, '3': 0b11111111, '4': 0b00000111,
        '5': 0b00101011, '6': 0b11111100, '7': 0b00011111, '8': 0b10000011,
        '9':0b10010101, '10': 0b11110010, '11': 0b10111110, '12': 0b01010111,
        '13': 0b11111000, '14': 0b00111000, '15': 0b00100111, '16': 0b11100000,
        '17': 0b11111100, '18': 0b00011111, '19': 0b10000011, '20': 0b11110000,
        '21': 0b01111110, '22': 0b00001111, '23': 0b11000001, '24': 0xFF, '25': 0xFF}

data1 = {'A': -50, 'E':0, 'T':-100, 'R': 0,
        '1': 0x0F, '2': 0b11011101, '3': 0b10111111, '4': 0b00000111,
        '5': 0b00101011, '6': 0b11111100, '7': 0b00011111, '8': 0b10000011,
        '9':0b10010101, '10': 0b11110010, '11': 0b10111110, '12': 0b01010111,
        '13': 0b11111000, '14': 0b00111000, '15': 0b00100111, '16': 0b11100000,
        '17': 0b11111100, '18': 0b00011111, '19': 0b10000011, '20': 0b11110000,
        '21': 0b01111110, '22': 0b00001111, '23': 0b11000001, '24': 0xFF, '25': 0xFF}

data1 = {'A': 50, 'E':0, 'T':-100, 'R': 0,
        '1': 0x0F, '2': 0b10100001, '3': 0b01011111, '4': 0b00000111,
        '5': 0b00101011, '6': 0b11111100, '7': 0b00011111, '8': 0b10000011,
        '9':0b10010101, '10': 0b11110010, '11': 0b10111110, '12': 0b01010111,
        '13': 0b11111000, '14': 0b00111000, '15': 0b00100111, '16': 0b11100000,
        '17': 0b11111100, '18': 0b00011111, '19': 0b10000011, '20': 0b11110000,
        '21': 0b01111110, '22': 0b00001111, '23': 0b11000001, '24': 0xFF, '25': 0xFF}


data1 = {'A': 100, 'E':0, 'T':-100, 'R': 0,
        '1': 0x0F, '2': 0b11110111, '3': 0b00011111, '4': 0b00000111,
        '5': 0b00101011, '6': 0b11111100, '7': 0b00011111, '8': 0b10000011,
        '9':0b10010101, '10': 0b11110010, '11': 0b10111110, '12': 0b01010111,
        '13': 0b11111000, '14': 0b00111000, '15': 0b00100111, '16': 0b11100000,
        '17': 0b11111100, '18': 0b00011111, '19': 0b10000011, '20': 0b11110000,
        '21': 0b01111110, '22': 0b00001111, '23': 0b11000001, '24': 0xFF, '25': 0xFF}


In [ ]:

# Data from oscilloscope, after passed the uart and the inverter
data_1_inv = {'A': 0, 'E':3, 'T':-100, 'R': -2,
        '1': 0x0F, '2': 0b01101100, '3': 0b00110111, '4': 0b00000100,
        '5': 0b00000111, '6': 0b11110001, '7': 0x1F, '8': 0b11111101,
        '9':0xFF, '10': 0b10111111, '11': 0b11110111, '12': 0b11111110,
        '13': 0b11111111, '14': 0b11011111, '15': 0b11111011, '16': 0xFF,
        '17': 0b01111111, '18': 0b11101111, '19': 0b11111101, '20': 0xFF,
        '21': 0b10111111, '22': 0b11110111, '23': 0b11111110, '24': 0xFF, '25': 0xFF}

data_1_x8r = {'A': 0, 'E':3, 'T':-100, 'R': -2,
        '1': 0x0F, '2': 0b11000100, '3': 0b00111111, '4': 0b00000111,
        '5': 0b00101011, '6': 0b11111100, '7': 0x1F, '8': 0x83,
        '9':0x95, '10': 0xF2, '11': 0xBE, '12': 0x57,
        '13': 0xA6, '14': 0xA2, '15': 0xDB, '16': 0xE0,
        '17': 0xFC, '18': 0x1F, '19': 0x83, '20': 0xF0,
        '21': 0x7E, '22': 0x0F, '23': 0xC1, '24': 0xFF, '25': 0xFF}


In [ ]:
print(int(0b11101001000), -100)
print(int(0b10101011111), -50)
print(int(0b01011111010), 50)
print(int(0b11101101), 100)

import matplotlib.pyplot as plt
import numpy as np

a,b = np.linalg.solve([[-100, 1], [100, 1]], [1811, 172])
print(a, b)
print(-100*a+b)

fig, ax = plt.subplots(1,1)
ax.scatter(-99,1870)
ax.scatter(-50,1467)
ax.scatter(0, 1059)
ax.scatter(50,645)
ax.scatter(100,239)
#ax.axis('equal')

plt.show()

In [ ]:
import numpy as np

ch = [1811, 3]

buff = np.zeros(2, dtype=np.uint8)

buff[0] = ch[0] & 0x07FF
buff[1] = (ch[0] & 0x07FF) >> 8 | (ch[1] & 0x07FF) << 3

print(bin(buff[0]), bin(buff[1]))
print(buff[0].tobytes())
print(buff[1].tobytes())
print(buff.tobytes())


ch = [172, 3]

buff = np.zeros(2, dtype=np.uint8)

buff[0] = ch[0] & 0x07FF
buff[1] = (ch[0] & 0x07FF) >> 8 | (ch[1] & 0x07FF) << 3

print(bin(buff[0]), bin(buff[1]))
print(buff[0].tobytes())
print(buff[1].tobytes())
print(buff.tobytes())

ch = [0, 3]

buff = np.zeros(2, dtype=np.uint8)

buff[0] = ch[0] & 0x07FF
buff[1] = (ch[0] & 0x07FF) >> 8 | (ch[1] & 0x07FF) << 3

print(bin(buff[0]), bin(buff[1]))
print(buff[0].tobytes())
print(buff[1].tobytes())
print(buff.tobytes())

In [ ]:
import overpy

api = overpy.Overpass()

upper_left = {'LAT': 60.1871, 'LON': 24.8113}
lower_right = {'LAT': 60.1837, 'LON': 24.8245}

lat_southern_edge = lower_right['LAT']
lon_western_edge = upper_left['LON']
lat_northern_edge = upper_left['LAT']
lon_easthern_edge = lower_right['LON']

query_str = "nwr(" + str(lat_southern_edge) + "," + str(lon_western_edge) + "," + str(lat_northern_edge) + "," + str(lon_easthern_edge) + ");out;"

result = api.query(query_str)

In [ ]:
import os

print(os.getcwd() + '\\' + "map.html")


In [ ]:
import folium
import numpy as np
import pandas as pd

center_point = [40, -90]

data = (
    np.random.normal(size=(100, 2)) *
    np.array([[.5, .5]]) +
    np.array([center_point])
)

df = pd.DataFrame(data, columns=['Lat', 'Long'])
   
sw = df[['Lat', 'Long']].min().values.tolist()
ne = df[['Lat', 'Long']].max().values.tolist()

print(sw)
print(ne)

In [ ]:
from a2gmeasurements import RepeatTimer
import time

cnt = 0
def my_print(msg):
    global cnt
    cnt = cnt + 1
    print('ESTE ES MY MSG: ', msg, 'CNT: ', cnt)

tm = RepeatTimer(0.1, my_print, args=('ESTOESTO',))

start_time = time.time()
tm.start()

In [ ]:
tm.cancel()
stop_time = time.time()

print('INTERVAL: ', stop_time-start_time, ', EXEC_PER_FCN: ', (stop_time-start_time)/cnt)

In [ ]:
import datetime


"".join([str(i) + '-' for i in datetime.datetime.utcnow().timetuple()[0:3]])

In [ ]:
import pandas as pd

data = pd.read_csv('test_coordinates_map_micronova.csv', names=['LATITUDE', 'LONGITUDE'], sep=',')
data

In [ ]:
from pyproj import Geod
a = data.to_numpy()
b = a[1:, :]
c = a[:-1, :]

print(a)

n_stops_per_edge = 4
wgs84_geod = Geod(ellps='WGS84')
        
az12, az21, dist = wgs84_geod.inv(a[:-1, 1], a[:-1, 0], a[1:, 1], a[1:, 0])
print('DISTS: ', dist)
print('ANGS: ', az12)

intermediate_coords = {}
for i in range(b.shape[0]):
    intermediate_coords[str(i+1)] = {'LAT': [], 'LON': []}
    lon_2, lat_2, _ = wgs84_geod.fwd(c[i, 1], c[i, 0], az12[i], dist[i]/(n_stops_per_edge-1))
    intermediate_coords[str(i+1)]['LAT'].append(lat_2)
    intermediate_coords[str(i+1)]['LON'].append(lon_2)
    print('EDGE ', i, ': ',  lat_2, lon_2)
    for j in range(n_stops_per_edge-3):
        lon_2, lat_2, _ = wgs84_geod.fwd(lon_2, lat_2, az12[i], dist[i]/(n_stops_per_edge-1))
        intermediate_coords[str(i+1)]['LAT'].append(lat_2)
        intermediate_coords[str(i+1)]['LON'].append(lon_2)
        print('EDGE ', i, j, ': ',  lat_2, lon_2)
    
print(intermediate_coords)


In [ ]:
from a2gUtils import GpsOnMap
from matplotlib.figure import Figure
import time

hi_q = {'LAT': 60.18592, 'LON': 24.81174 }
mygpsonmap = GpsOnMap('planet_24.81,60.182_24.829,60.189.osm.pbf', air_coord=hi_q)

for i in range(20):
    mygpsonmap.show_air_moving() 
    time.sleep(0.5)


In [ ]:
import subprocess, re 

ifconfig_info = subprocess.Popen(["ifconfig"], stdout=subprocess.PIPE)
out, err = ifconfig_info.communicate()
stdout_str = out.decode()

# This is how the IP address should appear, as the GND-DRONE connection is wireless through an AP
stdout_str_split = stdout_str.split('wlan0: ')

pattern = r'inet\s+\d+.\d+.\d+.\d+'
gnd_ip_addr = re.findall(pattern, stdout_str_split[-1])
gnd_ip_addr = gnd_ip_addr[0].split('inet ')
gnd_ip_addr = gnd_ip_addr[-1]

gnd_ip_addr